In [1]:
import sys
import os
import subprocess
import numpy as np
from datetime import datetime, timedelta

sys.path.append('../sdpm_py_util')
import init_funs_forecast as initfuns
import run_funs as runfuns
sys.path.append('/home/mspydell/models/PFM_root/PFM/driver')
#from driver_run_forecast_LV1234 import driver_run_fore_LV1234
from driver_run_pfm_phm import driver_run_pfm_phm


/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


In [ ]:
infile = '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/pfm_input_file.py'
pkl_fnm = '/scratch/PFM_Simulations/forecast_pfile.pkl'


t00 = datetime.now()
# upon initialization, make the model_info.pkl file
initfuns.initialize_model( infile , pkl_fnm )

# print info from pickle file
initfuns.print_initial_model_info( pkl_fnm )


In [ ]:

driver_run_fore_LV1234( infile, pkl_fnm )

# to run from the command line do:
# python driver_run_hind_LV123.py '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/phm_model_info_devel.py' '/scratch/PHM_Simulations/hindcast_pfile.pkl' > hindcast.out


In [2]:
#run = 'hind'
run = 'fore'
if run == 'hind': # hindcasting works with this newer method (6/10/25). make sure you are in PHM-env
    infile = '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/phm_model_info_devel.py'
    pkl_fnm = '/scratch/PHM_Simulations/hindcast_pfile.pkl'
else:
    infile = '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/pfm_input_file.py'
    pkl_fnm = '/scratch/PFM_Simulations/forecast_pfile.pkl'



In [3]:
driver_run_pfm_phm( infile, pkl_fnm )


first, we need all the model information.
from /home/mspydell/models/PFM_root/PFM/sdpm_py_util/pfm_input_file.py
we create a dictionary of model info
forecast
the model_info dictionary is saved to /scratch/PFM_Simulations/forecast_pfile.pkl
this file dictates how the model is run, where files are saved, etc.
------------------------------------------------------------
                        Model Set Up                        
------------------------------------------------------------
forecast                                                    
from           : 2025-06-20 12:00                           
to             : 2025-06-25 12:00                           
in a           : 5.0 day chunk                              
atm model      : ecmwf                                      
ocean_model    : hycom_new                                  
running levels : LV1, LV2, LV3, LV4                         

starting LV1


/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


we are going to start clean...
getting PFM info...
cleaning out directories...
now making a new PFM.pkl file.
202506201200

Going to do a PFM forecast from
2025-06-20 12:00:00
to
2025-06-25 12:00:00


getting the hycom forecast time...
this will clean, download new hycom files, then find the forecast date covering the PFM times.
we currently have hycom forecasts starting from:
['2025-06-18', '2025-06-19']
checking to see if we are missing any files...
we are missing 263 files from these forecasts.
attempting to get these files ...
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
g

/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


we are now putting the fore atm data on the roms LV2 grid...
loading /scratch/PFM_Simulations/LV1_Forecast/Forc/atm_tmp_pckl_file.pkl ...
interpolating to 2 grid...
... done.
rotating velocities to LV 2 roms directions...
... done.
saving to 2 pkl file.

ATM on roms grid dict saved with pickle.
return code: 0 (0=good)
...done.
this took:
0:00:03.656562


we are now saving ATM LV2 to /scratch/PFM_Simulations/LV2_Forecast/Forc/LV2_ATM_FORCING.nc ...
return code: 2 (0=good)
...done.
this took:
0:00:00.008454


driver_run_forcast_LV2: saving LV2_OCN_BC pickle file
return code: 2 (0=good)
driver_run_forecast_LV2:  done with writing LV2_OCN_BC.pkl file.
this took:
0:00:00.008356


driver_run_forcast_LV2: saving LV2_OCN_BC netcdf file
return code: 2 (0=good)
driver_run_forecast_LV2:  done with writing LV2_OCN_BC.nc file.
this took:
0:00:00.007775


going to use a restart file for the LV2 IC. Setting this up...


python: can't open file '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/atm_functions_forecast.py': [Errno 2] No such file or directory
python: can't open file '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_functions_forecast.py': [Errno 2] No such file or directory
python: can't open file '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_functions_forecast.py': [Errno 2] No such file or directory


going to restart LV2 from
2025-06-20 12:00:00
['/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506190000_202506240000.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506160000_202506210000.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506130600_202506180600.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506170000_202506220000.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506140000_202506190000.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506131200_202506181200.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506150000_202506200000.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506200000_202506250000.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506181200_202506231200.nc']
found the time index!
going to restart using /scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506200000_202506250000.nc and python-index 1
PFM info was edited and resaved
...done setting up f

Traceback (most recent call last):
  File "/home/mspydell/models/PFM_root/PFM/driver/driver_functions.py", line 1125, in <module>
    globals()[args[1]](*args[2:])
  File "/home/mspydell/models/PFM_root/PFM/driver/driver_functions.py", line 1115, in run_fore_simulation
    run_fore_LV2(pkl_fnm)
  File "/home/mspydell/models/PFM_root/PFM/driver/driver_functions.py", line 1069, in run_fore_LV2
    runfuns.run_slurm_LV2( pkl_fnm )
TypeError: run_slurm_LV2() missing 1 required positional argument: 'mod_type'


done with LV2
LV2  forecast ran correctly? 1 (0=yes)
this took 

starting LV3
done with LV3
LV3  forecast ran correctly? 1 (0=yes)
this took 

starting LV4
done with LV4
LV4  forecast ran correctly? 0 (0=yes)
this took 


Traceback (most recent call last):
  File "/home/mspydell/models/PFM_root/PFM/driver/driver_functions.py", line 1125, in <module>
    globals()[args[1]](*args[2:])
  File "/home/mspydell/models/PFM_root/PFM/driver/driver_functions.py", line 1117, in run_fore_simulation
    run_fore_LV3(pkl_fnm)
    ^^^^^^^^^^^^
NameError: name 'run_fore_LV3' is not defined. Did you mean: 'run_fore_LV1'?


In [ ]:
print('making dot in dot sb ...')
runfuns.make_LV1_dotin_and_SLURM( pkl_fnm , 'fore' )
print('...done')
print('running roms ...')
runfuns.run_slurm_LV1( pkl_fnm , 'fore')
print('...done')



making dot in dot sb ...
 --- making dot_in and dot_sb --- 
loading model info...
...done
for this LV1 simulation
history file made will be:
/scratch/PFM_Simulations/LV1_Forecast/His/LV1_ocean_his_20250617.nc
restart file made will be:
/scratch/PFM_Simulations/restart_data/LV1_ocean_rst_20250617_20250622.nc
the ini file used is:
/scratch/PFM_Simulations/restart_data/LV1_ocean_rst_202506170000_202506220000.nc
we are using
/scratch/PFM_Simulations/executables/LV3_romsM_INTEL
...done
running roms ...
run_slurm_LV1: current directory is now:  /scratch/PFM_Simulations/LV1_Forecast/Run
CompletedProcess(args=['sbatch', '--wait', 'LV1_SLURM.sb'], returncode=0, stdout=b'Submitted batch job 2125\n', stderr=b'')
subprocess slurm ran correctly? 0 (0=yes)
run_slurm_LV1: run command:  ['sbatch', '--wait', 'LV1_SLURM.sb']
...done


In [ ]:
os.chdir('../driver')
cmd_list = ['python','driver_functions.py','run_fore_simulation','LV2',pkl_fnm]
ret1 = subprocess.run(cmd_list)     
 


/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


we are now putting the fore atm data on the roms LV2 grid...
loading /scratch/PFM_Simulations/LV1_Forecast/Forc/atm_tmp_pckl_file.pkl ...
interpolating to 2 grid...
... done.
rotating velocities to LV 2 roms directions...
... done.
saving to 2 pkl file.

ATM on roms grid dict saved with pickle.
return code: 0 (0=good)
...done.
this took:
0:00:03.720532


we are now saving ATM LV2 to /scratch/PFM_Simulations/LV2_Forecast/Forc/LV2_ATM_FORCING.nc ...
return code: 0 (0=good)
...done.
this took:
0:00:01.242789


driver_run_forcast_LV2: saving LV2_OCN_BC pickle file
return code: 2 (0=good)
driver_run_forecast_LV2:  done with writing LV2_OCN_BC.pkl file.
this took:
0:00:00.008967


driver_run_forcast_LV2: saving LV2_OCN_BC netcdf file
return code: 2 (0=good)
driver_run_forecast_LV2:  done with writing LV2_OCN_BC.nc file.
this took:
0:00:00.008583


going to use a restart file for the LV2 IC. Setting this up...


python: can't open file '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_functions_forecast.py': [Errno 2] No such file or directory
python: can't open file '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_functions_forecast.py': [Errno 2] No such file or directory


going to restart LV2 from
2025-06-20 12:00:00
['/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506190000_202506240000.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506160000_202506210000.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506130600_202506180600.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506170000_202506220000.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506140000_202506190000.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506131200_202506181200.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506150000_202506200000.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506200000_202506250000.nc', '/scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506181200_202506231200.nc']
found the time index!
going to restart using /scratch/PFM_Simulations/restart_data/LV2_ocean_rst_202506200000_202506250000.nc and python-index 1
PFM info was edited and resaved
...done setting up f

Traceback (most recent call last):
  File "/home/mspydell/models/PFM_root/PFM/driver/driver_functions.py", line 1125, in <module>
    globals()[args[1]](*args[2:])
  File "/home/mspydell/models/PFM_root/PFM/driver/driver_functions.py", line 1115, in run_fore_simulation
    run_fore_LV2(pkl_fnm)
  File "/home/mspydell/models/PFM_root/PFM/driver/driver_functions.py", line 1069, in run_fore_LV2
    runfuns.run_slurm_LV2( pkl_fnm )
TypeError: run_slurm_LV2() missing 1 required positional argument: 'mod_type'
